In [1]:
%run ../Python_files/load_dicts.py

In [2]:
%run ../Python_files/util.py

In [3]:
cd '/home/jzh/Dropbox/Research/Data-driven_estimation_inverse_optimization/Experiments/InverseVIsTraffic/temp_files/'

/home/jzh/Dropbox/Research/Data-driven_estimation_inverse_optimization/Experiments/InverseVIsTraffic/temp_files


In [4]:
link_1_1, link_1_2, link_2_1, link_2_2, link_3_1, link_3_2, \
	link_4_1, link_4_2, link_5_1, link_5_2, link_6_1, link_6_2, \
	link_7_1, link_7_2, link_8_1, link_8_2, link_9_1, link_9_2, \
	link_10_1, link_10_2, link_11_1, link_11_2, link_12_1, link_12_2 \
    = zload('../temp_files/links_with_capac.pkz')

In [5]:
# 0 <---> link_1_1, 1 <---> link_1_2, 2 <---> link_2_1, 3 <---> link_2_2
# 4 <---> link_3_1, 5 <---> link_3_2, 6 <---> link_4_1, 7 <---> link_4_2
# 8 <---> link_5_1, 9 <---> link_5_2, 10 <---> link_6_1, 11 <---> link_6_2
# 12 <---> link_7_1, 13 <---> link_7_2, 14 <---> link_8_1, 15 <---> link_8_2
# 16 <---> link_9_1, 17 <---> link_9_2, 18 <---> link_10_1, 19 <---> link_10_2
# 20 <---> link_11_1, 21 <---> link_11_2, 22 <---> link_12_1, 23 <---> link_12_2

link_with_capac_list = list(zload('links_with_capac.pkz'))

In [6]:
len(link_with_capac_list)

24

In [7]:
len(tmc_capac_dict_AM)

701

In [8]:
link_with_capac_list[0]

In [9]:
link_1_1.AM_flow

0

In [10]:
tmc_ref_speed_dict['129+04099']

60.0

In [11]:
road_seg_inr_capac = zload('../temp_files/road_seg_inr_capac.pkz')

In [12]:
len(road_seg_inr_capac.tmc)

701

In [13]:
# load tmc-day-ave_speed data for AM peak of July
tmc_day_speed_dict_Jul_AM = zload('Jul_AM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for MD of July
tmc_day_speed_dict_Jul_MD = zload('Jul_MD/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for PM peak of July
tmc_day_speed_dict_Jul_PM = zload('Jul_PM/tmc_day_speed_dict.pkz')

# load tmc-day-ave_speed data for NT of July
tmc_day_speed_dict_Jul_NT = zload('Jul_NT/tmc_day_speed_dict.pkz')

In [14]:
tmc_day_speed_dict_Jul_AM['129+040994'].ave_speed()

59.412825955124305

In [15]:
tmc_day_speed_dict_Jul_NT['129+040991'].ave_speed()

56.982949577382719

In [16]:
tmc_day_capac_flow_dict = {}

for i in range(len(road_seg_inr_capac.tmc)):
    for day in range(32)[1:]:
        tmc = road_seg_inr_capac.tmc[i]
        road_num = road_seg_inr_capac.road_num[i]
        shape_length = road_seg_inr_capac.shape_length[i]
        day = day
        AB_AM_capac = road_seg_inr_capac.AB_AM_capac[i]
        AB_MD_capac = road_seg_inr_capac.AB_MD_capac[i]
        AB_PM_capac = road_seg_inr_capac.AB_PM_capac[i]
        AB_NT_capac = road_seg_inr_capac.AB_NT_capac[i]
        AM_ave_speed = tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed()
        MD_ave_speed = tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed()
        PM_ave_speed = tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed()
        NT_ave_speed = tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed()
        tmc_day_capac_flow = RoadSegInrCapacFlow(tmc, road_num, shape_length, day, \
                                                  AB_AM_capac, AB_MD_capac, \
                                                  AB_PM_capac, AB_NT_capac, \
                                                  AM_ave_speed, MD_ave_speed, \
                                                  PM_ave_speed, NT_ave_speed)
        tmc_day_capac_flow_dict[tmc + str(day)] = tmc_day_capac_flow

In [17]:
len(tmc_day_capac_flow_dict)

21731

In [18]:
zdump(tmc_day_capac_flow_dict, 'tmc_day_capac_flow_dict_Jul.pkz')

In [19]:
tmc_day_capac_flow_dict['129+04099' + '4']

In [20]:
tmc_day_capac_flow_dict['129+04099' + '7'].PM_flow()

2223.1040021112931

In [21]:
len([tmc_day_capac_flow_dict[tmc + str(day)].AM_flow() for tmc in link_12_1.tmc_set])

11

In [22]:
len(link_12_1.tmc_set)

11

In [23]:
link_1_2.free_flow_time

0.32860120689201772

In [24]:
link_with_capac_list[0].AM_flow

0

In [25]:
link_day_Jul_dict = {}

for day in range(32)[1:]:
    i = 0
    for link_with_capac in link_with_capac_list:
        AM_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].AM_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_AM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        MD_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].MD_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_MD[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        PM_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].PM_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_PM[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        NT_flow = sum([tmc_day_capac_flow_dict[tmc + str(day)].NT_flow() * tmc_length_dict[tmc] / \
                       tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set]) / \
                  sum([tmc_length_dict[tmc] / tmc_day_speed_dict_Jul_NT[tmc + str(day)].ave_speed() \
                       for tmc in link_with_capac.tmc_set])
        link_with_capac_new = Link_with_Free_Flow_Time(link_with_capac.init_node, link_with_capac.term_node, \
                                                       link_with_capac.tmc_set, \
                                                       link_with_capac.AM_capac, \
                                                       link_with_capac.MD_capac, \
                                                       link_with_capac.PM_capac, \
                                                       link_with_capac.NT_capac, \
                                                       link_with_capac.free_flow_time, \
                                                       link_with_capac.length, \
                                                       AM_flow, MD_flow, PM_flow, NT_flow)
        link_day_Jul_dict['link_' + str(i) + '_' + str(day)] = link_with_capac_new
        i = i + 1

zdump(link_day_Jul_dict, 'link_day_Jul_dict.pkz')

In [26]:
link_day_Jul_dict['link_22_5'].length / link_day_Jul_dict['link_22_5'].free_flow_time

71.64839271583233

In [30]:
link_day_Jul_dict['link_22_9'].PM_flow

1328.5887091115103